In [ ]:
import os
import sys
import tqdm

from collections import defaultdict
import json
import re

import matplotlib.pyplot as plt

sys.path.insert(0, '..')
from src import emojilib

# torch
# torch
import torch
from torch.utils.data import Dataset

## Config

In [ ]:
root_path = './'
clean_data_folder_path = os.path.join(root_path, 'data', 'clean_data')
tweet_text_path = os.path.join(clean_data_folder_path, 'tweet_by_ID_30_4_2022__08_11_07.txt.text')
tweet_id_path = os.path.join(clean_data_folder_path, 'tweet_by_ID_30_4_2022__08_11_07.txt.ids')
tweet_label_path = os.path.join(clean_data_folder_path, 'tweet_by_ID_30_4_2022__08_11_07.txt.labels')

# target emojis
mapping = { 
    '❤':'0' , '😍':'1' , '😂':'2' , '💕':'3' , 
    '🔥':'4' , '😊':'5' , '😎':'6' , '✨':'7' , 
    '💙':'8' , '😘':'9' , '📷':'10' , '🇺🇸':'11' , 
    '☀':'12' , '💜':'13' , '😉':'14' , '💯':'15' , 
    '😁':'16' , '🎄':'17' , '📸':'18' , '😜':'19'
}

## Dataset Class [WIP]

In [ ]:
class EmojiDataset(Dataset):
    def __init__(self, dataset_path, transforms=None):
        tweet_text_path = os.path.join(dataset_path, 'tweet_by_ID_30_4_2022__08_11_07.txt.text')
        tweet_label_path = os.path.join(dataset_path, 'tweet_by_ID_30_4_2022__08_11_07.txt.labels')
        
        sentences = []
        labels = []
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        
        